# Overview
A conceptual model for the problem would be:
\begin{array}{rl}
    \text{min} & \text{unused volume in used bins} \\
    \text{subject to}   & \text{all items assigned to one and only one bin} \\
                        & \text{all items within the bin dimensions} \\
                        & \text{no overlaps between items in the same bin} \\
                        & \text{all items with support} \\
\end{array}

The model also need to account for 90° rotations on the same plane which can be modeled as swapping its width and depth.

# MILP Model
## Sets
\begin{array}{rl}
    I = \{1 \dots n \} & \text{set of items} \\
    J = \{1 \dots m \} & \text{set of homogenous bins} \\
\end{array}

## Parameters
\begin{array}{rll}
    W \times D \times H & \text{width $\times$ depth $\times$ height of a bin} & \\
    V & \text{bin volume} & \\
    w_i \times d_i \times h_i & \text{width $\times$ depth $\times$ height of an item} & \forall i \in I\\
    \alpha_s & \text{support coefficient} & \\
    \beta_s & \text{support tollerance} & \\
\end{array}

## Variables
\begin{array}{rll}
    v_{ij} & \begin{cases}
                1, \text{if item $i$ is in bin $j$} \\ 
                0, \text{otherwise}
            \end{cases} & \forall i \in I,j \in J \\
    u_j & \begin{cases}
                1, \text{if bin $j$ is used} \\ 
                0, \text{otherwise}
            \end{cases} & \forall j \in J \\
    (x_i,y_i,z_i) & \text{bottom front left corner of an item} & \forall i \in I \\
    (x'_i,y'_i) & \text{back right corner of an item} & \forall i \in I \\
    (x^{\text{max}}_{ik},y^{\text{max}}_{ik}) & \text{the maximum top back right coordinate along each axis between items $i,k$} & \forall i,k \in I \\
    (x^{\text{min}}_{ik},y^{\text{min}}_{ik}) & \text{the minimum bottom front left coordinate along each axis between items $i,k$} & \forall i,k \in I \\
    r_i & \begin{cases}
                1, \text{if item $i$ is rotated 90° over its z-axis} \\ 
                0, \text{otherwise}
            \end{cases} & \forall j \in J \\
    x^p_{ik} & \begin{cases}
                1, \text{if $x_i \le x'_k$} \\ 
                0, \text{otherwise}
            \end{cases} & \forall i,k \in I \\
    y^p_{ik} & \begin{cases}
                1, \text{if $y_i \le y'_k$} \\ 
                0, \text{otherwise}
            \end{cases} & \forall i,k \in I \\
    z^p_{ik} & \begin{cases}
                1, \text{if $z_i \le z_k + h_k$} \\ 
                0, \text{otherwise}
            \end{cases} & \forall i,k \in I \\
    z^c_{ik} & \begin{cases}
                1, \text{if $0 \le z_k - (z_i + h_i) \le \beta_s$ } \\ 
                0, \text{otherwise}
            \end{cases} & \forall i,k \in I \\
    s_{ik} & \begin{cases}
                1, \text{if item $i$ offers support for item $k$} \\ 
                0, \text{otherwise}
            \end{cases} & \forall i,k \in I \\
    (t^x_{ik}, t^y_{ik}) & \text{the overlap edge length between item $i,k$ } & \forall i,k \in I \\
    z^\text{max}_j & \text{maximum height of bin $j$} & \forall j \in J
\end{array}

Where $i,k \in I, j \in J$ and $x_i,y_i,z_i,x'_i,y'_i \in \mathbb{Z}^+$.

The definition of the point $x'_i,y'_i$ of an item is mostly used to facilitate the concept of rotations.

An overlap on axis $a$ occurs if $a^p_{ik} \land a^p_{ki}$

## Objective Function
\begin{array}{rll}
    \text{min} & \sum_j^m{u_jH + z^\text{max}_j} & \text{(1)} \\
\end{array}

(1) minimizes the number of opened bins

## Constraints
\begin{array}{lll}
    \sum_j^m{v_{ij}} = 1, & \forall i \in I & (2) \\
    v_{ij} \le u_j, & \forall i \in I,j \in J & (3) \\
    x'_i - x_i = (1 - r_i)w_i + r_i d_i & \forall i \in I & (4) \\
    y'_i - y_i = (1 - r_i)d_i + r_i w_i & \forall i \in I & (5) \\
    x'_i \le W & \forall i \in I & (6) \\
    y'_i \le D & \forall i \in I & (7) \\
    z_i \le H - h_i & \forall i \in I & (8) \\
\end{array}

(2) Ensures that an item is assigned to one and only one bin.

(3) in conjunction with (1) ensures that the least ammount of bins is used and a bin is used only if at least one item is in it.

(4,5) define the concept of rotating an item 90° over its z-axis.

(6,7,8) ensure that all the items are within the bin

\begin{array}{lll}
    x_i - x'_k < (1 - x^p_{ik})W & \forall i,k \in I & (9) \\
    x'_k - x_i \le x^p_{ik}W & \forall i,k \in I & (10) \\
    y_i - y'_k < (1 - y^p_{ik})D & \forall i,k \in I & (11) \\
    y'_k - y_i \le y^p_{ik}D & \forall i,k \in I & (12) \\
    z_i - (z_k + h_k) < (1 - z^p_{ik})H & \forall i,k \in I & (13) \\
    z_k + h_k - z_i \le z^p_{ik}W & \forall i,k \in I & (14) \\
    x^p_{ik} + x^p_{ki} + y^p_{ik} + y^p_{ki} + z^p_{ik} + z^p_{ki} \le 7 - (v_{ij} + v_{kj}) & \forall (i,k)  \in I, j \in J &  (15)
\end{array}

(9-14) fully define the precedence variables on each axis, and ensure that if on axis $a$ items $i,k$ are such that $a_i = a'_k$ then $a^p_{ik} = 0$.

(15) ensures that overlap is only present at maximum on 2 axis for items in the same bin. An overlap occurs if i precedes k and k precedes i over an axis.

\begin{array}{lll}
    s_{ik} \le \frac{v_{ij} + v_{kj}}{2} & \forall (i,k) \in I, j \in J & (16) \\
    s_{ik} \le (1 - z^p_{ik}) & \forall (i,k) \in I, j \in J & (17) \\
    s_{ik} \le z^c_{ki} & \forall (i,k) \in I, j \in J & (18) \\
    s_{ik} \ge v_{ij} + v_{kj} + (1 - z^p_{ki}) + z^c_{ki} - 3 & \forall (i,k) \in I: i \ne k & (19) \\
    \beta_s - (z_i - (z_k + h_k)) \le  z^c_{ki}H & \forall (i,k) \in I: i \ne k & (20) \\
\end{array}

(16-19) ensures that $i$ can give support to $k$ if and only if $v_ij \land v_kj \land (1 - z^p_{ik}) \land z^c_{ki}$

(20) ensures that $i$ is considered "close" to $k$ only when $z_i + h_i$ is within tollerance of $z_k$ 

\begin{array}{lll}
    x^{\text{max}}_{ik} \le x'_i & \forall i,k \in I & (21) \\
    x^{\text{max}}_{ik} \le x'_k & \forall i,k \in I & \\
    x^{\text{min}}_{ik} \ge x_i & \forall i,k \in I & (22) \\
    x^{\text{min}}_{ik} \ge x_k & \forall i,k \in I & \\
    y^{\text{max}}_{ik} \le y'_i & \forall i,k \in I & (23) \\
    y^{\text{max}}_{ik} \le y'_k & \forall i,k \in I & \\
    y^{\text{min}}_{ik} \ge y_i & \forall i,k \in I & (24) \\
    y^{\text{min}}_{ik} \ge y_k & \forall i,k \in I & \\
    t^x_{ik} \le x^{\text{max}}_{ik} - x^{\text{min}}_{ik} & \forall i,k \in I & (25) \\
    t^y_{ik} \le y^{\text{max}}_{ik} - y^{\text{min}}_{ik} & \forall i,k \in I & (26) \\
    x^{\text{max}}_{ik}, x^{\text{min}}_{ik}, y^{\text{max}}_{ik}, y^{\text{min}}_{ik}, t^x_{ik}, t^y_{ik} \ge 0 & \forall i,k \in I & \\
    \sum^n_k s_{ki} t^x_{ik} t^y_{ik} \ge \alpha_s w_i d_i & \forall i \in I & (28)
\end{array}

(28) guarantees support for each item $i$, the constraint is not linear


\begin{array}{lll}
    z^\text{max}_j \ge z_i + h_i - (1 - v_{ij})H & \forall i \in I, j \in J & (29) \\
    z^\text{max}_j \ge 0 & \forall j \in J &
\end{array}

(29) in conjuction with (1) ensures that solutions with the same ammount of opened bins are scored by maximum height of their items

### Notes
- Add an image explaining the coordinate system and parameters

### Paper references
- The Pallet Loading Problem: Three-dimensional Bin Packing with Practical Constraints (Fatma Gzara, Samir Elhedhli, Burak C. Yildiz) (concept of support)
- A mixed integer programming formulation for the three-dimensional bin packing problem deriving from an air cargo application (C. Paquay, M. Schyns and S. Limbourg) (nice formulation of the 3DBPP)